# YOLO9000: Better, Faster, Stronger

paper(16.12) https://arxiv.org/abs/1612.08242<br>
code https://pjreddie.com/darknet/yolov2/

---

# Abstract

**We introduce YOLO9000, a state-of-the-art, real-time object detection system that can detect over 9000 object categories.**<br>
YOLO의 두 번째 버전인 YOLO9000은 9000개의 클래스를 구분할 수 있다고 자신하기 때문에 이러한 이름을 붙였다.

**"Fisrt we propose various improvements to the YOLO detection method, both novel and drawn from prior work."**<br>
기존 YOLO 모델을 토대로 새로운 방법들을 제안한다. 적용한 Dataset마다 performace를 수치화함으로써 입증하고 있다.

**"Using a novel, multi-scale training method the same YOLOv2 model can run at varying sizes, offering an easy tradeoff between speed and accuracy."**<br>
YOLOv2(YOLO9000과 사실상 동일한 모델)는 multi-scale training method를 통해 다양한 사이즈의 크기를 학습하여 그 성능을 더 높였다.

**"Finally we propose a method to jointly train on object detection and classification"**<br>
해당 논문은 두 개의 dataset을 합쳐서 학습을 시키는 독특한 접근을 하는데, 나는 이에 대한 이해가 완벽하지는 않아서 간단히 살펴보고 넘어갈 예정이다.

# 1. Introduction

**"We propose a new method to harness the large amount of clasification data we already have and use it to expand the scope of current detection system. Our method uses a hierarchical view of object classification that aloows us to combine distinct datasets together."**<br>
해당 논문은 classification dataset과 detection dataset의 결합을 획기적인 방법으로 소개하고 있다. 이전 단락에서도 언급한 바 있지만, 이는 classification dataset에 비해 상대적으로 적은 양의 detection dataset을 효과적으로 사용할 수 있는 방법이다. 

**"We also propose a joint training algorithm that allows us to train object detection on both detection and classification data. Our method leverages labeled detection images to learn to precisely localize objects while it uses classifcation images to increase its vocabulary and robustness"**<br>
내 생각엔 위의 단락과 비슷한 맥락의 설명으로 보인다. 다만 좀 더 아이디어를 명확히 했는데, 라벨링된 detection image를 통해 물체를 localization하는 방법을 배우고, classification image를 통해 어휘력과 견고성을 높일 수 있다고 언급한다. 결국 핵심은 classifcation dataset과 detection dataset의 결합과 학습이라고 생각한다.

# 2. Better

**"Thus we focus mainly on improving recall and localization while maintaining classification accuracy."**<br>
기존 YOLO의 문제점은 다른 SOTA 모델들보다 localization error가 높게 나타난다. 게다가 Faster R-CNN과 같은 region proposal-based method 에 비해 상대적으로 낮은 recall값을 갖는다. 따라서 YOLO v2는 이러한 점에 중점을 두어 모델을 보완했다고 볼 수 있다.

**"We pool a variety of ideas from past work with our own novel concepts to improve YOLO's performance"**<br>
YOLO의 성능을 향상시키기 위해 기존에 연구되어있던 아이디어들을 접목시켜 모델을 보완했다. 아래와 같이 표를 통해 그 결과를 입증하고 있으며, 적용한 아이디어들을 하나씩 소개한다.

<img src='Image/YOLOv201.PNG' width='100%'>

### Batch Normalization

* 2% improvement in mAP
* Help regularize the model
* We can remove dropout from the model without overfitting

### High Resolution Classifier

* Fine tune the classification network at the full 448 x 448 resolution for 10 epochs on ImageNet
* Then fine tune the resulting network on detection
* An increase of almost 4% mAP

기존 YOLO 모델은 classifier network를 224 x 224 image로 학습한 뒤 detection을 위해 448 x 448로 resolution을 증가시켰다. 이로 인해 모델이 새로운 input size에 적응해야만 하기 때문에 좋은 성능을 기대할 수 없었다. 따라서 YOLO v2는 classifier network를 448 x 448 image로 10번의 fine tuning 후에 detection 부분을 학습함으로써, 모델이 higer resolution input에 적응할 수 있는 시간을 주었다.

### Convolutional With Anchor Boxes

* Remove the fully connected layers from YOLO and use anchor boxes to predict bounding boxes
* Eliminate one pooling layer to make the output of the network's convolutional layers higher resolution

YOLOv2는 YOLO와 달리 FC layer들을 모두 제거하고, Faster R-CNN이 사용하는 anchor box의 개념을 도입하였다. 이는 Image에서 object를 더 잘 찾아내기 위한 하나의 방법이라고 생각한다.

* Shrink the network to operate on 416 input images instead of 448 x 448

input image의 크기를 448에서 416으로 줄였다. YOLOv2는 5번의 downsampling을 수행하는데, 만약 448 크기라면 14 x 14의 feature map을 갖는 반면 416 크기라면 13 x 13의 feature map을 갖는다. 이는 일반적으로 큰 물체가 image의 중앙에 위치하는 특성을 반영한 것으로, 홀수 크기의 feature map을 구상함으로써 중앙에 위치한 물체를 더 잘 인식하도록 하기 위함이다. 만약 14 x 14와 같이 짝수 크기의 feature map이라면, 중앙에 있는 물체를 감지하는 grid cell은 총 4개가 되어 정확도가 떨어질 것이다.

* Predict class and objectness for every anchor box

YOLO v1의 경우 grid cell별로 class predicting을 수행했지만, 이를 수정하여 grid cell의 anchor box별로 class predicting을 수행하도록 바꾸었다. 즉, anchor box별로 class와 objectness predicting을 모두 수행한다.

**"Even though the mAP decreases, the increase in recall means that our model has more room to imporve"**<br>

Category | Without anchor box | With anchor box
:---:|---:|---:
mAP | 69.5 | 69.2
recall | 81% | 88%

anchor box를 도입하면 그러지 않은 경우보다 mAP가 살짝 감소하지만, recall은 증가하는 결과를 볼 수 있다. recall이 증가한다는 것은 모델이 더 성장할 가능성이 있다는 것으로 YOLO v2는 anchor box method를 채택한다.

### Dimension Clusters

To Do~